### Apriori Method

---


In [47]:
from collections import Counter
import numpy as np
from itertools import combinations
import pandas as pd

In [48]:
dataset = pd.read_csv('garments_worker_productivity.csv')
dataset['department'].unique()

array(['sweing', 'finishing ', 'finishing'], dtype=object)

In [49]:
dataset = [
    ["T100", ["I1", "I2", "I5"]],
    ["T200", ["I2", "I4"]],
    ["T300", ["I2", "I3"]],
    ["T400", ["I1", "I2", "I4"]],
    ["T500", ["I1", "I3"]],
    ["T600", ["I2", "I3"]],
    ["T700", ["I1", "I3"]],
    ["T800", ["I1", "I2", "I3", "I5"]],
    ["T900", ["I1", "I2", "I3"]],
]

In [50]:
# Find List of Unique Items
unique_items = []
for i in dataset:
    for j in i[1]:
        if j not in unique_items:
            unique_items.append(j)
unique_items = sorted(unique_items)
print("Unique Itemset")
print(unique_items)

Unique Itemset
['I1', 'I2', 'I3', 'I4', 'I5']


In [51]:
# Find Minimum Support Count
support_percentage = 0.5  # 50%
min_sup = int(support_percentage * len(unique_items))
print("Minimum Support Count is -> ", min_sup)

Minimum Support Count is ->  2


In [52]:
# Generate Itemsets
def generate_itemsets(dataset, unique_items):
    C = Counter()
    # 1 candidate itemset
    for i in unique_items:
        for j in dataset:
            if i in j[1]: # iterate unique itemset and dataset to check if it exists
                C[i] += 1
    print("C1:")
    for i in C:
        print(str([i]) + " -> " + str(C[i]))
    L = Counter()
    for i in C:
        if C[i] >= min_sup:
            L[frozenset([i])] += C[i]
    print("L1:")
    for i in L:
        print(str(list(i)) + " -> " + str(L[i]))
    print("\n")
    
    # possible sets of items
    final_itemset = L
    set_no = 1
    for count in range(2, 1000):
        new_set = set()
        u = list(L)
        for i in range(0, len(u)):
            for j in range(i + 1, len(u)):
                new_set.add(u[i].union(u[j])) # create combination of current itemsets    
        new_set = list(new_set)
        
        # n possible candidate itemsets
        C = Counter()
        for i in new_set:
            C[i] = 0
            for j in dataset:
                u = set(j[1])
                if i.issubset(u): # count occurence of each itemset and print
                    C[i] += 1
        if(len(C) == 0):
            break
        print("C" + str(count) + ":")
        for i in C:
            print(str(list(i)) + " -> " + str(C[i]))
        L = Counter()
        for i in C:
            if C[i] >= min_sup: # filter out needed candidates
                L[i] += C[i]
        print("L" + str(count) + ":")
        for i in L:
            print(str(list(i)) + " -> " + str(L[i]))
        print("\n") 
        if len(L) == 0:
            break
        final_itemset = L
        set_no = count
    print("Final Itemsets:")
    print("L" + str(set_no) + ":")
    for i in final_itemset:
        print(str(list(i)) + " -> " + str(final_itemset[i]))
    return final_itemset
final_itemset = generate_itemsets(dataset, unique_items)

C1:
['I1'] -> 6
['I2'] -> 7
['I3'] -> 6
['I4'] -> 2
['I5'] -> 2
L1:
['I1'] -> 6
['I2'] -> 7
['I3'] -> 6
['I4'] -> 2
['I5'] -> 2


C2:
['I4', 'I3'] -> 0
['I4', 'I2'] -> 2
['I1', 'I4'] -> 1
['I2', 'I3'] -> 4
['I5', 'I2'] -> 2
['I5', 'I3'] -> 1
['I1', 'I2'] -> 4
['I1', 'I5'] -> 2
['I5', 'I4'] -> 0
['I1', 'I3'] -> 4
L2:
['I4', 'I2'] -> 2
['I2', 'I3'] -> 4
['I5', 'I2'] -> 2
['I1', 'I2'] -> 4
['I1', 'I5'] -> 2
['I1', 'I3'] -> 4


C3:
['I5', 'I4', 'I2'] -> 0
['I1', 'I5', 'I4', 'I2'] -> 0
['I1', 'I4', 'I2', 'I3'] -> 0
['I4', 'I2', 'I3'] -> 0
['I1', 'I4', 'I2'] -> 1
['I1', 'I5', 'I2', 'I3'] -> 1
['I1', 'I5', 'I3'] -> 1
['I1', 'I2', 'I3'] -> 2
['I5', 'I2', 'I3'] -> 1
['I1', 'I5', 'I2'] -> 2
L3:
['I1', 'I2', 'I3'] -> 2
['I1', 'I5', 'I2'] -> 2


C4:
['I5', 'I2', 'I1', 'I3'] -> 1
L4:


Final Itemsets:
L3:
['I1', 'I2', 'I3'] -> 2
['I1', 'I5', 'I2'] -> 2


In [53]:
# Find Minimum Confidence
min_conf_percent = 0.4  # 40%
print("Minimum Confidence is -> ", min_conf_percent)

Minimum Confidence is ->  0.4


In [55]:
for l in final_itemset:
    c = [frozenset(q) for q in combinations(l,len(l)-1)]
    mmax = 0
    for a in c:
        b = l-a
        ab = l
        sab = 0
        sa = 0
        sb = 0
        for q in dataset:
            temp = set(q[1])
            if(a.issubset(temp)):
                sa+=1
            if(b.issubset(temp)):
                sb+=1
            if(ab.issubset(temp)):
                sab+=1
        temp = sab/sa*100
        if(temp > mmax):
            mmax = temp
        temp = sab/sb*100
        if(temp > mmax):
            mmax = temp
        print(str(list(a))+" -> "+str(list(b))+" = "+str(sab/sa*100)+"%")
        print(str(list(b))+" -> "+str(list(a))+" = "+str(sab/sb*100)+"%")

['I1', 'I2'] -> ['I3'] = 50.0%
['I3'] -> ['I1', 'I2'] = 33.33333333333333%
['I1', 'I3'] -> ['I2'] = 50.0%
['I2'] -> ['I1', 'I3'] = 28.57142857142857%
['I2', 'I3'] -> ['I1'] = 50.0%
['I1'] -> ['I2', 'I3'] = 33.33333333333333%
['I1', 'I5'] -> ['I2'] = 100.0%
['I2'] -> ['I1', 'I5'] = 28.57142857142857%
['I1', 'I2'] -> ['I5'] = 50.0%
['I5'] -> ['I1', 'I2'] = 100.0%
['I5', 'I2'] -> ['I1'] = 100.0%
['I1'] -> ['I5', 'I2'] = 33.33333333333333%
